In [1]:
import tensorflow as tf
import numpy as np


In [5]:
#卷积：
#参数为  名字 前两个表示过滤器尺寸，第三个是当前层深度 第四个是过滤器深度（也就是过滤器个数，也是下层的深度）
filter_writer = tf.get_variable('weights',[5,5,3,16],initializer = tf.truncated_normal_initializer(stddev = 0.1)) 
biases = tf.get_variable('biases',[16],initializer = tf.constant_initializer(0.1))
#tf.nn.conv2d计算卷积
#第一个参数是batch   input[0, , , ]表示第一章图片 input[1, , , ]表示第二张图片 。。。
#第二个参数是bias  
#第三个参数是一个长度为4的数组 格式为[1, , ,1] 表示filter对输入的操作
#第四个参数是填充方法（padding） 为'SAME' （全0）和 'VALID'（不添加）
conv = tf.nn.conv2d(input_,filter_weights,strides = [1,1,1,1],padding = 'SAME')
#由于每一次层的节点偏置项相同，利用tf.nn.bias_add函数
output = tf.nn.bias_add(conv,biases)
#计算结果用relu激活
active_conv = tf.nn.relu(output)

#池化 (池化过滤器只影响一个深度的节点)
#平均值或者最大值池化
#tf.nn.max_pool  tf.nn.avg_pool  也有padding参数





NameError: name 'input_' is not defined

In [2]:
#LeNet-5模型实现识别mnist
from tensorflow.examples.tutorials.mnist import input_data

#一些经验：一般下一层卷积的filter数会乘2

#训练参数：
BATCH_SIZE = 100#随机梯度下降法
learning_rate_origin = 0.8 #初始学习率
learning_rate_decay = 0.99#衰减率
lambda_ = 0.0001#正则化lambda
train_srep = 30000#训练轮次
moving_average_decay = 0.99

#卷积池化卷积池化 再用一个卷积层（此层的过滤器只有一个）把输出铺平  全连接

#先建立placehoder
#x = tf.placeholder(tf.float32,[BATCH_SIZE,len_,width_,deepth],name = 'x-input')#RGB格式深度为5

#模型参数
INPUT_NODE = 784 #28*28
OUTPUT_NODE = 10
IMAGE_SIZE = 28 #图片尺寸
NUM_CHANNELS = 1 #
NUM_LABELS = 10  #标签数量
#第一层卷积
CONV1_DEEP = 32
CONV1_SIZE = 5
#第二层卷积
CONV2_DEEP = 64
CONV2_SIZE = 5
#全连接节点个数
FC_SIZE = 512




Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
def inference(input_tensor, train, regularizer):
    # 声明第一层卷积层的变量并实现前向传播过程
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable(
            'weight', [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable(
            'bias', [CONV1_DEEP], initializer=tf.constant_initializer(0.0))

        # 使用边长为5，深度为32的过滤器，过滤器步长为1，使用全0填充
        conv1 = tf.nn.conv2d(
            input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    # 实现第二层池化层的前向传播。使用最大池化层，池化层过滤器的边长为2，步长为2，使用全0填充
    with tf.variable_scope('layer2-pool1'):
        pool1 = tf.nn.max_pool(
            relu1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 声明第三层卷积层的变量并实现前向传播
    with tf.variable_scope('layer3-conv2'):
        conv2_weights = tf.get_variable(
            'weight', [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable(
            'bias', [CONV2_DEEP], initializer=tf.constant_initializer(0.0))

        # 使用边长为5，深度为64的过滤器，过滤器步长为1，使用全0填充
        conv2 = tf.nn.conv2d(
            pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    # 实现第四层池化层的前向传播
    with tf.variable_scope('layer4-pool2'):
        pool2 = tf.nn.max_pool(
            relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    #得到pool2层的矩阵大小
    # 将第四层池化层的输出转化为第五层全连接层的输入格式
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    reshaped = tf.reshape(pool2, [pool_shape[0], nodes])

    # 声明第五层全连接层的变量并前向传播
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable(
            'weight', [nodes, FC_SIZE],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        fc1_biases = tf.get_variable(
            'bias', [FC_SIZE], initializer=tf.constant_initializer(0.1))

        # 只有全连接层的权重加入正则化
        if regularizer != None:
            tf.add_to_collection('losses', regularizer(fc1_weights))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)

        # 只有训练的时候才使用dropout
        if train:
            fc1 = tf.nn.dropout(fc1, 0.5)

    # 声明第六层的变量并前向传播
    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable(
            'weight', [FC_SIZE, NUM_LABELS],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        fc2_biases = tf.get_variable(
            'bias', [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        if regularizer != None:
            tf.add_to_collection('losses', regularizer(fc2_weights))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases

    return logit #返回向前传播结果

import os

# 优化方法参数
LEARNING_RATE_BASE = 0.01  # 基础学习率
LEARNING_RATE_DECAY = 0.99  # 学习率的衰减率
REGULARIZATION_RATE = 0.0001  # 正则化项在损失函数中的系数
MOVING_AVERAGE_DECAY = 0.99  # 滑动平均衰减率

# 训练参数
BATCH_SIZE = 100  # 一个训练batch中的图片数
TRAINING_STEPS = 30000  # 训练轮数

# 模型保存的路径和文件名
MODEL_SAVE_PATH = 'model/'
MODEL_NAME = 'lenet5.ckpt'


def train(mnist):
    # 实现模型
    x = tf.placeholder(
        tf.float32, [
            BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE,
            NUM_CHANNELS
        ],
        name='x-input')  # 输入层
    y_ = tf.placeholder(
        tf.float32, [None, OUTPUT_NODE], name='y-input')  # 标签
    regularizer = tf.contrib.layers.l2_regularizer(
        REGULARIZATION_RATE)  # 定义L2正则化损失函数
    y = inference(x, True, regularizer)  # 输出层

    # 存储训练轮数，设置为不可训练
    global_step = tf.Variable(0, trainable=False)

    # 设置滑动平均方法
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)  # 定义滑动平均类
    variable_averages_op = variable_averages.apply(
        tf.trainable_variables())  # 在所有可训练的变量上使用滑动平均

    # 设置指数衰减法
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY)

    # 最小化损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))  # 计算每张图片的交叉熵
    cross_entropy_mean = tf.reduce_mean(cross_entropy)  # 计算当前batch中所有图片的交叉熵平均值
    loss = cross_entropy_mean + tf.add_n(
        tf.get_collection('losses'))  # 总损失等于交叉熵损失和正则化损失的和
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        loss, global_step=global_step)  # 优化损失函数

    # 同时反向传播和滑动平均
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化持久化类
    #saver = tf.train.Saver()

    # 开始训练
    with tf.Session() as sess:
        # 初始化所有变量
        tf.global_variables_initializer().run()

        # 迭代训练
        for i in range(TRAINING_STEPS):
            # 产生该轮batch
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            xs = np.reshape(
                xs, (BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE,
                     NUM_CHANNELS))  # 将MNIST数据格式转为四维矩阵
            _, loss_value, step = sess.run(
                [train_op, loss, global_step], feed_dict={
                    x: xs,
                    y_: ys
                })

            # 每1000轮保存一次模型
            if i % 1000 == 0:
                # 输出训练情况
                print('After %d training steps, loss is %g.' % (step,
                                                                loss_value))

                # 保存当前模型
                #saver.save(
                #    sess,
                #   os.path.join(MODEL_SAVE_PATH, MODEL_NAME),
                #    global_step=global_step)


# 主程序入口
def main(argv=None):

    train(mnist)


if __name__ == '__main__':
    tf.app.run()
#主程序
def main(argv = None):
    mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
    train(mnist)

if __name__ =='__main__':
     tf.app.run()

After 1 training steps, loss is 4.94549.
After 1001 training steps, loss is 0.876469.
After 2001 training steps, loss is 0.800536.


In [29]:
len(mnist.test.images)


10000